<h1><center><u> Code Implementation </u></center></h1>

# Core Thesis Work: Missing Data Imputation on Numerical DHS Dataset
---
---
### Code Implementation Walkthrough

This notebook walks through the full implementation used in the main experimental part of the thesis, focusing on missing data imputation for **purely numerical DHS and synthetic datasets**.

> • **Goal**: Demonstrate missing data imputation with *purely numerical* DHS and synthetic datasets using a modular, scalable pipeline.
>
> • **Code Folder**: All core logic for data loading, imputation and evaluation is organized in the `src/` directory for clarity and reusability.
>
> • **Notebook Purpose**: Designed for running and visualizing the full experimental pipeline. This notebook complements the *Main Experimental Study* section of the thesis by supporting exploratory data analysis (EDA) and final metric evaluation by running the full experimental pipeline.
>
> • **Explore**: See `src/preprocessing.py`, `src/imputations.py`, `src/evaluation.py`, and `src/utils.py` for detailed implementation of the core components.

---

## Methodology Overview

This project uses a range of classical, machine learning and deep learning methods to impute missing data in real-world and synthetic numerical datasets. Steps are structured to ensure consistency, fairness and clarity in evaluation.

---

## Flowchart Overview

Below is a visual representation of the full pipeline used for the thesis:

<img src="Flowchart_thesis_25_april.jpg" alt="Methodology Flowchart" width="900"/>

---

## Key Steps in the Methodology

#### 1. Data Preprocessing
- **Source:** Data collected from the DHS Program Website.
- **Steps Involved:**
  1. Initial extensive data preprocessing to clean and organize the data.
  2. Filtering data to remove irrelevant or redundant entries.
  3. Exploratory Data Analysis (EDA) to understand missingness patterns and distributions are analyzed during EDA.

#### 2. Dataset Preparation

- **Handling Missing Values:**
  - Three strategies:
    1. Dropping all NaNs for a more restrictive dataset (Complete Dataset or **'drop_all_nans'** asked in prompt).
    2. Dropping entire survey groups (column named 'Meta; GEID_init' having several survey IDs) that have more than 20% missing data or (**'numerical_only_drop_20_percentage_nans'** asked in prompt).
    3. Using initial KNN Imputation for initial filling of NaNs in the entire numerical dataset (**'numerical_only'** asked in prompt).

- **Cross-Validation:**
  - K-Fold cross-validation splits Data into training, validation and test folds.
  - Standard scaling is applied to ensure consistent feature ranges.

#### 3. Model Training and Validation

- **Two types of Missingness Introduced:**
  - MAR (Missing At Random) and MCAR (Missing Completely At Random).
- **Pre-Imputation:**
  - Initial KNN imputation is used to preprocess data for autoencoders (only for AE, DAE, and VAE) as these can not be trained on a dataset with NaNs.
- **Training Methods:**
  - Statistical, Machine learning and Deep learning methods used:
    - Mean, KNN, MICE Ridge
    - Autoencoders (AE)
    - Denoising Autoencoders (DAE)
    - Variational Autoencoders (VAE)
    - GAIN
  - Methods are trained using either:
    1. Datasets with artificial missing values (Mean, KNN, MICE Ridge, GAIN).
    2. Pre-KNN Imputed datasets after artificial missingness creation (for Autoencoders only).

#### 4. Hyperparameter Tuning and Testing

- Methods are tuned using the training folds and validated on separate test folds to ensure unbiased final evaluation.
- Final evaluation metrics are recorded under **both MAR and MCAR** scenarios.

#### 5. Final Evaluation

- **Performance Metrics:** 

  - RMSE (Root Mean Square Error)
  - MAE (Mean Absolute Error)
  - R² (R-squared)
  - Pearson Correlation
  - Results are compared across methods to determine the most effective approach.

The methodology ensures robust data handling and model evaluation for missing data imputation. The flowchart visually represents each step, highlighting the workflow and key decisions made in the study.

---

### Conclusion

This end-to-end pipeline ensures reliable and consistent evaluation of imputation techniques on numerical DHS data. With reusable components and clear structure, the notebook supports both experimentation and reproducibility.

# Libraries and Setup

Importing necessary libraries for data manipulation, statistical methods, machine learning, deep learning and evaluation.

In [ ]:
#importing required libraries

import gc
import os
import sys
import json
import time
import joblib
import pickle
import random
import string
import itertools
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from icecream import ic
import tensorflow as tf
from scipy import stats
from tabulate import tabulate
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from keras.regularizers import l2
from scipy.stats import spearmanr
from tensorflow.keras import Model
from collections import defaultdict
from itertools import combinations
import tensorflow.keras.backend as K
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from tensorflow.keras.losses import mse
from scipy.stats import levene, f_oneway
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import mse as keras_mse
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tensorflow.keras.layers import Lambda, Layer, Input, Dense
from sklearn.linear_model import BayesianRidge, LinearRegression
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, GaussianNoise, Layer
from tensorflow.keras import Sequential, layers, models, optimizers, regularizers
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures, FunctionTransformer

sys.path.append(os.path.abspath(".."))
import src.config_loader
importlib.reload(src.config_loader)

%load_ext autoreload
%autoreload 2

from src.utils import *
from src.setup_dirs import *
from src.evaluation import *
from src.run_pipeline import *
from src.visualization import *
from src.preprocessing import *
from src.config_loader import *
from src.gain import GAIN_code_v2
from src.helper_functions import *
from src.single_imputation import *
from src.load_data import load_data
from src.initial_imputation import *
from src.deep_learning_methods import *
from src.synthetic_data_generation import *
from src.config_loader import load_config, find_project_root
from src.dhs_modelling_functions_new import final_ds_droping_cols, fold_generator

# initialization
from pandarallel import pandarallel
pandarallel.initialize()

In [ ]:
!pip install openpyxl

## setting up random seeds for reproducibility

In [ ]:
tf.random.set_seed(6688)
random.seed(6688)
np.random.seed(6688)

## GPU Configuration

Configuring TensorFlow to use GPUs and manage memory efficiently.

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        raise e

#use multiple GPUs
gpus = [0] # may request more, if necessary
gpus = ["GPU:" + str(i) for i in gpus]
# https://keras.io/guides/distributed_training/
print('gpus', gpus)

references from https://github.com/gheisenberg/FoodSecurity/tree/main/DHS

## Data Loading and Configuration with directory creation

Setting up configurations for real and synthetic data handling, including masking and directory setup.

In [ ]:
### loading base configuration settings (real/synthetic, directory paths etc.)
# for either real and (aggregated + numerical) DHS data or synthetic data
config = load_config()
# prompt user to select how missing values (NaNs) should be handled in the dataset
config = select_process_nans_option(config)
# prompt user to choose type of missingness simulation: MAR (with masking) or MCAR (without masking)
config['masking'] = select_masking_option()
# setting a label to be used in filenames and directory names based on the masking choice
config['output_file_prefix'] = 'with_masking' if config['masking'] else 'without_masking'
# setting up necessary directories for saving results, figures
config = setup_directories(config)
# extracting paths from the directory structure for task-specific outputs
masking_dir, final_imputations_dir_missing_ratio, feature_eval_dir, task3_time_dir = get_final_imputations_dir(config)
# loading the DHS or synthetic dataset according to the user-defined config settings
input_df, initial_missingness= load_data(config)

In [ ]:
input_df.shape

In [ ]:
input_df.head()

In [ ]:
print(input_df.columns)

# Exploratory Data Analysis (EDA) only for numerical DHS datasets
### set "use_synthetic_data": false - from config.json

In [ ]:
# only keeping numerical data with missing values
numerical_only = input_df.select_dtypes(include=[np.number])
numerical_only.shape

In [ ]:
numerical_only.isnull().sum()

In [ ]:
# removing prefixes from column names for better understanding
numerical_only.columns = numerical_only.columns.str.replace(r'^(DHS Num; |DHS Cat; )', '', regex=True)
# extracting the initial part of each column name
initial_columns = numerical_only.columns.str.extract(r'(^[^:]+)', expand=False)

# a mask to keep only the first occurrence of each initial part
unique_columns_mask = ~initial_columns.duplicated()

# the mask to keep only unique initial columns
unique_df = numerical_only.loc[:, unique_columns_mask]

# filtering the data to include only the years between 2006 and 2022
filtered_df = unique_df[(unique_df['Meta; year'] >= 2001) & (unique_df['Meta; year'] <= 2022)]

# calculating the proportion of missing data per year
missing_data_yr = filtered_df.groupby('Meta; year').apply(lambda x: x.isnull().mean())

# plotting the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(missing_data_yr, cbar=True, cmap='viridis', yticklabels=True, linecolor='black', linewidths=0.5)
plt.xlabel('Features', fontsize=14, labelpad=10)
plt.ylabel('Years', fontsize=14)
plt.title('Missing Data Heatmap Per Year (2001-2022)', fontsize=16, pad=15)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# saving the plot 
# get current working directory
current_dir = os.getcwd()
# defining output path in current directory
output_path = os.path.join(current_dir, "heatmap_per_year_missing_data.png")
# saving and showing the plot
plt.savefig(output_path, dpi=300)
plt.show()

## Pattern of Missing Data in DHS Dataset

In [ ]:
# setting up a larger figure with a custom color palette and aesthetic adjustments
plt.figure(figsize=(10, 13))  # adjusting size as needed for clarity

# customizing colors and add colorbar with label directly in sns.heatmap
sns.heatmap(
    unique_df.isnull(),
    cbar=False,
    cmap="cividis",
    yticklabels=False,  # removing row labels (sample numbers)
    # cbar_kws={'label': 'Missing Data (Yellow) / Complete Data (Blue)'}  # Add colorbar label here
)

# enhancing plot title, labels and colorbar for a professional presentation
plt.title("Pattern of Missing Data in DHS Dataset", fontsize=16, pad=15)
plt.xlabel("Features", fontsize=14, labelpad=10)
plt.ylabel("Samples", fontsize=14, labelpad=10)

# customized the tick parameters for readability
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# showing and saving the plot
plt.tight_layout(pad=3.0)  # ensures the plot fits well without clipping
output_path = os.path.join(current_dir, "all_rows_final_missing_data_pattern.png")
plt.savefig(output_path, dpi=300)  # high resolution for publication
plt.show()

## Initial KNN imputation to fill NaNs to retain most of the dataset size and diversity instead of dropping all NaNs

In [ ]:
# applying KNN Imputer
imputer = KNNImputer(n_neighbors=5)
df1_imputed = pd.DataFrame(imputer.fit_transform(numerical_only), columns=numerical_only.columns)

## Correlation Matrix for cleaned numerical_only dataset

In [ ]:
# thresholds for correlation categories
high_corr_threshold = 0.7
moderate_corr_threshold = 0.3

# Pearson and Spearman correlations
pearson_corr_matrix = df1_imputed.corr()
spearman_corr_matrix = df1_imputed.corr(method='spearman')

# flattening the matrices to prepare for analysis 
pearson_pairs = pearson_corr_matrix.unstack().reset_index()
spearman_pairs = spearman_corr_matrix.unstack().reset_index()
pearson_pairs.columns = ["Feature_X", "Feature_Y", "Pearson_Correlation"]
spearman_pairs.columns = ["Feature_X", "Feature_Y", "Spearman_Correlation"]

# merging Pearson and Spearman correlations into one df
correlation_df = pd.merge(pearson_pairs, spearman_pairs, on=["Feature_X", "Feature_Y"])

# exclude self-correlations
correlation_df = correlation_df[correlation_df["Feature_X"] != correlation_df["Feature_Y"]]

# categorize correlations by strength
total_pairs = len(correlation_df)
high_corr_count = len(correlation_df[correlation_df["Pearson_Correlation"].abs() > high_corr_threshold])
moderate_corr_count = len(correlation_df[(correlation_df["Pearson_Correlation"].abs() <= high_corr_threshold) & 
                                         (correlation_df["Pearson_Correlation"].abs() > moderate_corr_threshold)])
no_corr_count = len(correlation_df[correlation_df["Pearson_Correlation"].abs() <= moderate_corr_threshold])

# calculating percentages for linear relationships
high_corr_percentage = (high_corr_count / total_pairs) * 100
moderate_corr_percentage = (moderate_corr_count / total_pairs) * 100
no_corr_percentage = (no_corr_count / total_pairs) * 100

# calculating percentages for non-linear relationships using Spearman correlations
non_linear_count = len(correlation_df[correlation_df["Spearman_Correlation"].abs() < moderate_corr_threshold])
linear_count = total_pairs - non_linear_count
non_linear_percentage = (non_linear_count / total_pairs) * 100
linear_percentage = (linear_count / total_pairs) * 100

# printing the results
print(f"Highly correlated features (Pearson > {high_corr_threshold}): {high_corr_percentage:.2f}%")
print(f"Moderately correlated features (Pearson {moderate_corr_threshold} - {high_corr_threshold}): {moderate_corr_percentage:.2f}%")
print(f"No correlation (Pearson <= {moderate_corr_threshold}): {no_corr_percentage:.2f}%")
print(f"Non-linear relationships (Spearman < {moderate_corr_threshold}): {non_linear_percentage:.2f}%")
print(f"Linear relationships (Spearman >= {moderate_corr_threshold}): {linear_percentage:.2f}%")

# saving correlation data to CSV in the current directory
csv_save_path = os.path.join(current_dir, "pearson_spearman_correlation_analysis.csv")
correlation_df.to_csv(csv_save_path, index=False)

# plotting the heatmap for the top 20 correlated pairs (by Pearson correlation)
top_20_corr_pairs = correlation_df.nlargest(20, "Pearson_Correlation", keep='all')[["Feature_X", "Feature_Y"]]
top_features = list(set(top_20_corr_pairs["Feature_X"]).union(set(top_20_corr_pairs["Feature_Y"])))
top_corr_matrix = pearson_corr_matrix.loc[top_features, top_features]

# masking the upper triangle
mask = np.triu(np.ones_like(top_corr_matrix, dtype=bool))

# removing gridlines and plot the heatmap
plt.figure(figsize=(14, 12))
sns.set(style="white")
sns.heatmap(top_corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0, annot_kws={"size": 8}, 
            linewidths=.5, cbar_kws={"shrink": 0.8})
plt.title('Top Correlated Features', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

# saving the heatmap in the current directory
save_path = os.path.join(current_dir, "corr_matrix_original_numerical_only_dataset.png")
plt.savefig(save_path, bbox_inches='tight')
plt.show()

# Task1 and Task3: Final Evaluation for RMSE vs Missing Ratio

In [ ]:
# list of imputers
imputers = ['Mean', 'KNN', 'MICE_Ridge', 'AE', 'DAE', 'VAE', 'GAIN']

# the missing ratios
missing_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

# important columns 
important_columns = [
    'DHS Cat; source of drinking water: piped into dwelling',
    'DHS Num; cluster altitude in meters: mean',
    'DHS Num; number of mosquito bed nets: mean',
    'DHS Num; time to get to water source (minutes): mean',
    'DHS Cat; location of source for water: in own dwelling',
    'DHS Cat; type of toilet facility: flush to piped sewer system',
    'DHS Num; number of household members: mean',
    'DHS Cat; has mobile telephone: yes',
    'DHS Num; number of mosquito bed nets: mean',
    'DHS Cat; has television: yes',
    'DHS Cat; type of cooking fuel: lpg',
    'DHS Num; hectares of agricultural land (1 decimal): mean',
    'DHS Num; owns sheep: mean',
    'DHS Num; total adults measured: mean'
]

In [ ]:
# function to handle both with masking (mar) and without masking (mcar)
def handle_masking_and_evaluation(input_df, imputers, missing_ratios, config, masking):

    # loading or computing data and saving pickles
    load_or_compute_data_part(input_df, imputers, missing_ratios, config, config['masking'])
    
    # evaluating Metrics using Saved Pickles
    final_results, extracted_values, time_metrics = evaluate_metrics_part(input_df, imputers, missing_ratios, config)

    # plotting RMSE statistics
    missing_ratio_vs_stats(final_results, imputers, missing_ratios, config)

    plot_std_boxplot_for_ratio_30(final_results, imputers, config, missing_ratio=0.3)
    
    # saving time metrics to excel
    save_time_metrics_to_excel(time_metrics, config)

    # plotting time metrics (for visualizing training and test time per imputation method)
    plot_time_vs_missing_ratio(time_metrics, config)

    # scatter plots for the entire df
    combined_df_scatter_plots(extracted_values, config, missing_ratio=0.3)

    # per column scatter plots for the entire df
    per_column_scatter_plots(extracted_values, important_columns, imputers, config, missing_ratio=0.3)
    
handle_masking_and_evaluation(input_df, imputers, missing_ratios, config, config['masking'])

# Task2: Final Evaluation for RMSE vs Num of Features

In [ ]:
# defining the intervals for the number of features
feature_intervals = [15, 30, 45, 60, 75, 96]

# loading or computing data and saving pickles
load_or_compute_data_feature_evaluation(input_df, imputers, feature_intervals, config, config['masking'])

# evaluating metrics using saved pickles
final_results_features = evaluate_metrics_part_feature_evaluation(input_df, imputers, feature_intervals, config)

# running final pipeline for feature evaluaton and saving results
num_of_features_vs_stats(final_results_features, imputers, feature_intervals, config)